<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/coli/ERA5_Land_Daily_Aggregated_Mar_May_'16_'17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install earthengine-api --upgrade
# optional, but useful for interactive maps
!pip install geemap --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.2/463.2 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.24
    Uninstalling earthengine-api-1.5.24:
      Successfully uninstalled earthengine-api-1.5.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


# New Import Section


In [4]:
import ee, pandas as pd
import geemap

# Authenticate

In [14]:
ee.Authenticate()
ee.Initialize(project='data-scrapping-470316') #name ng project niyo sa earth engine

# click niyo yung left side ng malapit sa dp
# tapos project info
# then udner Cloud Project ayun i-copy paste


# Load ERA5-Land Daily Aggregated


In [50]:
# ERA5-Land Daily Aggregates dataset
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
  .filterDate('2016-03-01', '2017-05-31')

# Print The List of Bands


In [51]:
# Print available variables (bands)
bands = dataset.first().bandNames().getInfo()
print("Available Bands:", bands)

Available Bands: ['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_open_water_surfaces_excludi

# Manila Location

In [76]:
# Geometry Point
manila = ee.Geometry.Point([120.9842, 14.5995]).buffer(25000)

# Extract Daily Data


In [77]:
def extract(img):
    vals = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=manila,
        scale=10000,
        maxPixels=1e13
    )
    return ee.Feature(None, vals).set("date", img.date().format("YYYY-MM-dd"))

fc = dataset.map(extract)


# Fetch all bands + values

In [80]:
dates = fc.aggregate_array("date").getInfo()
var_names = dataset.first().bandNames().getInfo()
data = {v: fc.aggregate_array(v).getInfo() for v in var_names}

# Build Dataframe

In [81]:
df = pd.DataFrame({"date": dates})
for v in var_names:
    df[v] = data[v]

/tmp/ipython-input-3832312120.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-input-3832312120.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-input-3832312120.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ip

# Units Metadata + Concversions

In [82]:
metadata = []

for col in df.columns:
    if col == "date":
        continue

    original_name = col
    original_unit = "varies"
    converted_unit = original_unit
    new_name = col

    # Kelvin → Celsius
    if "temperature" in col:
        df[col] = df[col] - 273.15
        new_name = col + "_C"
        original_unit, converted_unit = "K", "°C"

    # Precipitation / Runoff / Evaporation (m → mm/day)
    elif col.endswith("_sum") or "precipitation" in col or "evaporation" in col or "runoff" in col:
        df[col] = df[col] * 1000
        new_name = col + "_mm"
        original_unit, converted_unit = "m", "mm/day"

    # Pressure (Pa → hPa)
    elif "pressure" in col:
        df[col] = df[col] / 100
        new_name = col + "_hPa"
        original_unit, converted_unit = "Pa", "hPa"

    # Wind (m/s → km/h)
    elif "u_component" in col or "v_component" in col or "wind" in col:
        df[col] = df[col] * 3.6
        new_name = col + "_kmh"
        original_unit, converted_unit = "m/s", "km/h"

    # Snow depth (m → cm)
    elif "snow_depth" in col:
        df[col] = df[col] * 100
        new_name = col + "_cm"
        original_unit, converted_unit = "m", "cm"

    # Radiation & Fluxes (J/m²/day → W/m²)
    elif col.endswith("_radiation_sum") or col.endswith("_flux_sum") or col.endswith("_heat_sum"):
        df[col] = df[col] / 86400
        new_name = col + "_Wm2"
        original_unit, converted_unit = "J/m²/day", "W/m²"

    # Rename
    df.rename(columns={col: new_name}, inplace=True)

    # Add metadata record
    metadata.append({
        "Band": original_name,
        "Converted Column": new_name,
        "Original Unit": original_unit,
        "Converted Unit": converted_unit
    })

In [66]:
# Build metadata DataFrame
meta_df = pd.DataFrame(metadata)

# Download CSV

In [84]:
from google.colab import files

filename = "ERA5-Land Daily Aggregated_MarMay_2016_2017_Manila.csv"
df.to_csv(filename, index=False)

# --- Download ---
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#

In [83]:
df.head()

,date,dewpoint_temperature_2m_C,temperature_2m_C,skin_temperature_C,soil_temperature_level_1_C,soil_temperature_level_2_C,soil_temperature_level_3_C,soil_temperature_level_4_C,lake_bottom_temperature_C,lake_ice_depth,...,v_component_of_wind_10m_min_kmh,v_component_of_wind_10m_max_kmh,surface_pressure_min_hPa,surface_pressure_max_hPa,total_precipitation_min_mm,total_precipitation_max_mm,leaf_area_index_high_vegetation_min,leaf_area_index_high_vegetation_max,leaf_area_index_low_vegetation_min,leaf_area_index_low_vegetation_max
0,2016-03-01,19.708187,24.720706,24.904391,25.882201,26.333545,26.707176,26.957534,23.715011,-2.069584e-18,...,-12.707170,-6.576510,1005.121345,1008.709504,0.005696,0.260315,3.134277,3.134744,2.517336,2.518900
1,2016-03-02,19.682245,23.863887,23.410615,24.553426,25.529860,26.636359,26.952836,22.964556,-2.252967e-18,...,-14.916068,-2.169259,1006.532020,1009.474167,0.004219,0.865615,3.134593,3.135048,2.518364,2.519915
2,2016-03-03,21.713915,24.623525,24.083164,24.836671,25.222247,26.508234,26.946280,23.222745,-2.196498e-18,...,-7.583926,-2.162124,1004.876800,1009.047654,0.016245,1.059806,3.134904,3.135410,2.519387,2.520960
3,2016-03-04,22.432780,24.870265,24.695197,25.165780,25.347142,26.397286,26.937769,23.465009,-3.113694e-18,...,-8.724921,0.479802,1003.690273,1006.874857,0.000131,0.596621,3.135267,3.135720,2.520425,2.522006
4,2016-03-05,22.729403,26.851906,27.133864,27.322442,26.518951,26.368707,26.928261,24.083087,-1.190082e-18,...,-2.210527,8.458519,1001.550948,1005.698979,-0.000004,0.027887,3.135577,3.136038,2.521471,2.522989
